In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*

In [ ]:
val model = Network("model")

// Create nodes
val source = Source(model, "Source")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
val fork = Fork(model, "Fork")
val join = Join(model, "Join", fork)
val sink = Sink(model, "Sink")

// Create job class
val jobclass1 = OpenClass(model, "class1")

In [ ]:
// Set arrival and service processes
source.setArrival(jobclass1, Exp(0.05))  // Arrival rate
queue1.setService(jobclass1, Exp(1.0))   // Service rate for first parallel branch
queue2.setService(jobclass1, Exp(2.0))   // Service rate for second parallel branch

In [ ]:
// Set routing matrix for fork-join topology
val P = model.initRoutingMatrix()
P.set(jobclass1, jobclass1, source, fork, 1.0)   // Source to Fork
P.set(jobclass1, jobclass1, fork, queue1, 1.0)   // Fork to Queue1 (parallel branch 1)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)   // Fork to Queue2 (parallel branch 2)
P.set(jobclass1, jobclass1, queue1, join, 1.0)   // Queue1 to Join
P.set(jobclass1, jobclass1, queue2, join, 1.0)   // Queue2 to Join
P.set(jobclass1, jobclass1, join, sink, 1.0)     // Join to Sink

model.link(P)

In [ ]:
// Solve with multiple solversval solvers = arrayOf(    JMT(model, "seed", 23000, "verbose", false, "keep", false),    MVA(model))for (solver in solvers) {    try {                val avgTable = solver.avgTable        avgTable.print()    } catch (e: Exception) {        println("Error with ${solver.name}: ${e.message}")    }}

This fork-join network demonstrates:

1. **Fork-Join Topology**: Jobs are split into parallel tasks and synchronized at completion
2. **Parallel Processing**: Two queues process forked tasks simultaneously
3. **Synchronization**: The Join node waits for both parallel tasks to complete
4. **Performance Analysis**: Shows how parallel processing affects system throughput

**Key Metrics**:
- **Source**: Job arrival point (arrival rate = 0.05)
- **Queue1**: Faster service (rate = 1.0)
- **Queue2**: Slower service (rate = 2.0, so mean service time = 0.5)
- **Join**: Synchronization point where jobs wait for both parallel tasks

**Expected Behavior**:
- Jobs arrive at rate 0.05
- Each job forks into 2 parallel tasks
- Queue2 is faster, so Queue1 becomes the bottleneck
- Join queue length reflects synchronization delays
- Overall throughput is limited by the slower parallel branch